In [1]:
!pip3 install numpy scikit-learn pandas
import pickle 
import pandas as pd
sparsevectors={}
with open('text_vectors.pkl', 'rb') as f:
    sparsevectors = pickle.load(f)


#dfdesc= pd.read_csv('nodes/nodes.tsv', sep='\t', header=0)

#dfdesc=dfdesc.set_index('id')


/tmp/ipykernel_30237/1583097034.py:6: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  sparsevectors = pickle.load(f)


# Use distance to add new edges to graph

In [6]:
from sklearn.metrics.pairwise import cosine_similarity

def distance_description(tup)->float:
    '''
    This method can be changed to any distance
    '''
    source=tup[0]
    target=tup[1]
    sp1=sparsevectors[source]
    sp2=sparsevectors[target]
    
    return cosine_similarity(sp1,sp2)[0][0]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 9.6 MB/s eta 0:00:000m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.3/502.3 KB 10.0 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 KB 8.9 MB/s eta 0:00:00m eta 0:00:01


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

dfdesc = dfdesc.fillna("")
corpus = dfdesc.values[:,0]

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

In [8]:
import pysparnn.cluster_index as ci

In [9]:
cp = ci.MultiClusterIndex(X, dfdesc.index)
with open('cluterIndex.bin', 'wb') as fh_out:
    pickle.dump(cp, fh_out)

/home/mavroudo/Desktop/EdgeRecomendation/env/lib/python3.10/site-packages/pysparnn/matrix_distance.py:192: RuntimeWarning: divide by zero encountered in divide
  magnitude = 1.0 / (a_root_sum_square * self.matrix_root_sum_square)


KeyboardInterrupt: 

### Load cluster index from file

In [2]:
with open('cluterIndex.bin', 'rb') as file_:
    cp2 =pickle.load(file_)

/tmp/ipykernel_30237/1472516460.py:2: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  cp2 =pickle.load(file_)


In [6]:
import warnings
warnings.filterwarnings("ignore")
def getsimilartups(idn):
    res=cp2.search(sparsevectors[idn], k=25, k_clusters=1, return_distance=True)
    tups=[ (iddd[1],idn) for iddd in res[0][1:] if iddd[0]<0.90]
    with open("new_edges.txt","a+") as f:
        for tup in tups:
            f.write(f"{tup[0]},{tup[1]}\n")
    

In [5]:
# Run it in a single thread
from tqdm import tqdm
def getsimilartupsSingle(idn):
    res=cp2.search(sparsevectors[idn], k=25, k_clusters=1, return_distance=True)
    tups=[ (iddd[1],idn) for iddd in res[0][1:] if iddd[0]<0.90]
    return tups

with open("new_edges.txt","w") as f:
    indexes = list(sparsevectors.keys())
    for i in tqdm(range(len(indexes))):
        idn = indexes[i]
        for tup in getsimilartupsSingle(idn):
            f.write(f"{tup[0]},{tup[1]}\n")
        
    

  0%|▌                                                                                                                                                             | 2965/836624 [00:38<3:01:30, 76.55it/s]


KeyboardInterrupt: 

In [8]:
from multiprocessing import Pool


with Pool(5) as p:
        data=p.map(getsimilartups, list(sparsevectors.keys()))


In [17]:
len(dfdesc.index)

837834

In [ ]:
import pickle 
with open('nnNodes.pkl', 'wb') as f:
    pickle.dump(data, f)